In [ ]:
import isambard

In [ ]:
col_pdb = isambard.ampal.convert_pdb_to_ampal('4bj3.pdb')

In [ ]:
col_pdb

In [ ]:
col_pdb.sequences

In [ ]:
ex_col = isambard.ampal.Assembly()
ex_col._molecules.extend(col_pdb[2:])

In [ ]:
ex_col

In [ ]:
ex_col.sequences

In [ ]:
for monomer in ex_col.get_monomers():
    if monomer.mol_code == 'HYP':
        del(monomer.atoms['OD1'])
        monomer.mol_code = 'PRO'
        monomer.mol_letter = 'P'
        monomer.is_hetero = False

In [ ]:
ex_col.sequences

In [ ]:
class CollagenExpParams(isambard.specifications.CoiledCoil):
    def __init__(self, aa=28, major_radius=5, major_pitch=85, p1=27, p2=27, p3=27, z_scale=1.0, eep1=0, eep2=0, auto_build=True):
        n = 3
        super().__init__(n, auto_build=False)
        self.aas = [aa] * n
        self.phi_c_alphas = [p1, p2, p3]
        self.minor_helix_types = ['PPII'] * n
        self.major_pitches = [major_pitch] * n
        self.major_radii = [major_radius] * n
        self.major_handedness = ['r'] * 3
        self.minor_repeats = [None] * 3
        rpr_ppii = 3.1
        self.z_shifts = [-rpr_ppii * (2 * z_scale), 0.0, -rpr_ppii * z_scale]
        rot_adj = [(z/p)*360 for z, p in zip(self.z_shifts, self.major_pitches)]
        self.rotational_offsets = [r+ra for r, ra in zip(self.rotational_offsets, rot_adj)]
        self.rotational_offsets[1] += eep1
        self.rotational_offsets[2] += eep2
        if auto_build:
            self.build()

In [ ]:
col_mod = CollagenExpParams(aa=16, major_radius=3.5, major_pitch=65)

In [ ]:
col_mod.rotational_offsets

In [ ]:
col_mod.pack_new_sequences([x[1:-1] for x in ex_col.sequences])

In [ ]:
col_mod.sequences

In [ ]:
isambard.external_programs.run_profit(ex_col.pdb, col_mod.pdb, path1=False, path2=False)

In [ ]:
sequences = col_mod.sequences

In [ ]:
sequences

This fitting required specific definition of the zone using the following script:

```
ignoremissing
align C*:A*
align D*:C* append
align E*:B* append
atoms ca
fit
atoms n,ca,c,o
fit
atoms *
fit
quit```

In [ ]:
opt = isambard.optimisation.DE_RMSD(
    CollagenExpParams, col_pdb.pdb)
opt.parameters(sequences,
          [3.5, 65, 0, 1.0, 0, 0],
          [2.0, 40, 180, 1, 30, 30],
          [16, 'var0', 'var1', 'var2', 'var2', 'var2', 'var3', 'var4', 'var5'])
opt.run_opt(30, 50, 3)

In [ ]:
best = CollagenExpParams(*opt.parse_individual(opt.halloffame[0]))
best.pack_new_sequences(sequences)